In [1]:
import pandas as pd
import glob
import numpy as np
from datetime import timedelta
from datetime import datetime
from collections import Counter
import statistics
import matplotlib.pyplot as plt

In [2]:
### IMPORT DATA
path1 = "../Desktop/raw_indiegogo_data/"

# import data
frames = []
for filename in glob.glob(path1 + "*"):
    print(filename)
    df = pd.read_json(filename, lines=True)
    frames.append(df)

../Desktop/raw_indiegogo_data/Indiegogo_2019-07-12T10_43_44_180Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-06-15T10_40_30_145Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-03-16T10_40_19_065Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2019-03-15T10_40_03_647Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2019-02-15T10_40_11_967Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-09-14T10_40_03_473Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-02-16T10_41_02_245Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-07-13T10_40_09_705Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-08-17T10_40_24_782Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-12-14T10_41_01_757Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2018-04-13T10_40_14_474Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2019-05-17T10_40_02_625Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2019-04-12T10_40_02_572Z.json
../Desktop/raw_indiegogo_data/Indiegogo_2019-01-18T10_40_13_766Z.json
../Desktop/raw_indie

In [3]:
df1 = pd.DataFrame()
df1 = pd.concat(frames, axis = 0, join = 'inner')

In [5]:
# set up dataframe for this additional data
project_id = []
created_at = []
deadline = []
category = []
goal = []
pledged = []
pg = []
state = []
country = []
in_demand = []

for i in df1['data']:
    if i['project_id'] != None and i['category'] != None and i['open_date'] != None and i['close_date'] != None and i['funds_raised_amount'] != None and i['funds_raised_percent'] != None and i['currency'] != None:
        project_id.append(i['project_id'])
        category.append(i['category'])
        str_open_date = i['open_date'][0:10]
        datetime_open_date = datetime.strptime(str_open_date, '%Y-%m-%d')
        created_at.append(datetime_open_date)
        str_close_date = i['close_date'][0:10]
        datetime_close_date = datetime.strptime(str_close_date, '%Y-%m-%d')
        deadline.append(datetime_close_date)
        pledged.append(i['funds_raised_amount'])
        if i['funds_raised_percent'] != 0:
            goal.append(i['funds_raised_amount'] / i['funds_raised_percent'])
        else:
            goal.append(-1)
        pg.append(i['funds_raised_percent'])
        if i['funds_raised_percent'] >= 1:
            state.append('successful')
        else:
            state.append('failed')
        country.append(i['currency'])
        in_demand.append(i['is_indemand'])

df_all = pd.DataFrame()
df_all['id'] = project_id
df_all['created_at'] = created_at
df_all['deadline'] = deadline
df_all['category'] = category
df_all['goal'] = goal
df_all['usd_pledged'] = pledged
df_all['P/G'] = pg
df_all['state'] = state
df_all['country'] = country
df_all['in_demand'] = in_demand

# convert epoch to datetime
df_all['created_at'] = pd.to_datetime(df_all['created_at'], unit = 's')
df_all['deadline'] = pd.to_datetime(df_all['deadline'], unit = 's')

# calculate duration of the project in months
df_all['duration'] = (df_all['deadline'] - df_all['created_at']) / np.timedelta64(1, 'M')

# convert date to month/year format for created at and deadline
df_all['Created At Month/Year'] = df_all['created_at'].apply(lambda x: "%d/%d" % (x.month, x.year))
df_all['Deadline Month/Year'] = df_all['deadline'].apply(lambda x: "%d/%d" % (x.month, x.year))

# convert date to month/year format for created at and deadline
df_all['Created At Year'] = df_all['created_at'].dt.year
df_all['Deadline Year'] = df_all['deadline'].dt.year

# only keep US companies
#df_all = df_all.loc[(df_all['country'] == 'USD')]

# remove duplicates
df_all = df_all.drop_duplicates('id')

# rearranging columns
df_all = df_all[['id','category','created_at','Created At Month/Year','Created At Year','deadline','Deadline Month/Year','Deadline Year',
         'duration','goal','usd_pledged', 'P/G','state','country', 'in_demand']]

df_minusgoal = df_all.loc[df_all['goal'] > 0]

In [7]:
print(df_all['country'])

0         USD
1         USD
2         USD
3         USD
4         USD
5         USD
6         USD
7         USD
8         USD
9         GBP
10        USD
11        EUR
12        USD
13        GBP
14        USD
15        USD
16        CAD
17        USD
18        USD
19        USD
20        USD
21        USD
22        USD
23        USD
24        USD
25        USD
26        GBP
27        USD
28        USD
29        USD
         ... 
529363    EUR
529364    USD
529365    USD
529369    EUR
529448    USD
529481    USD
529483    USD
529488    USD
529490    CAD
529501    USD
529506    USD
529507    USD
529508    USD
529510    USD
529511    USD
529520    USD
529521    USD
529540    USD
529546    USD
529553    USD
529557    USD
529579    USD
529596    USD
529661    CAD
529683    USD
529687    USD
529693    USD
529699    USD
529701    USD
529703    USD
Name: country, Length: 58611, dtype: object


In [11]:
### SAVE DATAFRAME

df_all.to_pickle('../Desktop/saved_dataframes/Store_indiegogo_dataframes.pkl')
df_minusgoal.to_pickle('../Desktop/saved_dataframes/Store_indiegogo_dataframes_minusgoal.pkl')

In [9]:
print(len(df_all))
print(len(df_minusgoal))

58611
53104
